In [ ]:
# 1. 解說題目
# 2. 30min , 說明靠打 API + 使用 headers , 介紹 Host & Referer
# 3. 1hr 解說 一次性 & 定期抓 差異
# 4. 抓資料, 解析

In [133]:
import requests
from bs4 import BeautifulSoup as bs 

# 必須要有 cookies & X-CSRF-TOKEN

### 一次性   --> 手動複製 cookies & X-CSRF-Token 即可

### 定期爬蟲 --> 須先向首頁發 requests 設定 cookies & 取得 csrf-token 

# 1. 建立 session 
# 2. sess 向首頁發 requests , 設定 cookies --> "https://rent.591.com.tw/"
# 3. 取得 csrf-token 
# 4. GET API 抓資料

base_url = "https://rent.591.com.tw/"

sess = requests.session()

headers0 = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"
}

res0 = sess.get(base_url,headers=headers0)
soup0 = bs(res0.text,"lxml")
token = soup0.select("meta[name='csrf-token']")[0]["content"]


url1 = "https://rent.591.com.tw/home/search/rsList?is_format_data=1&is_new_list=1&type=1&"

headers1 = {
    "Host": "rent.591.com.tw",
    "Referer": "https://rent.591.com.tw/?multiArea=10_20&showMore=1&section=3&searchtype=1&firstRow=0&totalRows=993",
    "X-CSRF-TOKEN": token,
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"
}

# res = requests.get(url,headers=headers1)
res = sess.get(url1,headers=headers1)
# print(res)
# print(res.json())

In [134]:
# list(sess.cookies)

In [73]:
##### 一次性抓資料 --> 手動設定 cookies & csrf-token
url2 = "https://rent.591.com.tw/home/search/rsList?is_format_data=1&is_new_list=1&type=1&section=3&searchtype=1&showMore=1&multiArea=10_20"

headers = {
    "Host": "rent.591.com.tw",
    "Referer": "https://rent.591.com.tw/?multiArea=10_20&showMore=1&section=3&searchtype=1&firstRow=0&totalRows=993",
    "Cookie": "webp=1; PHPSESSID=g3u3tnc81fjrk7ighgv2mkr3m4; urlJumpIp=1; urlJumpIpByTxt=%E5%8F%B0%E5%8C%97%E5%B8%82; new_rent_list_kind_test=1; newUI=1; T591_TOKEN=g3u3tnc81fjrk7ighgv2mkr3m4; _ga=GA1.3.1649913125.1636598035; _gid=GA1.3.822631898.1636598035; _ga=GA1.4.1649913125.1636598035; _gid=GA1.4.822631898.1636598035; tw591__privacy_agree=0; _fbp=fb.2.1636598035774.332302313; _gat_UA-97423186-1=1; XSRF-TOKEN=eyJpdiI6IlpSOXZ1SVlCZHpzNWswdzQzYThLUFE9PSIsInZhbHVlIjoiTGdzaXFPblN4bUZmQ3U1Z3AzWG1MZUNrbDFiMjIxMlNDamhUaWg5SVBFUUNZOEU0SHlzK1Z3S21walVDWWpHZFR4QVFmbEdrZmRqQnZyd0s2aitkRUE9PSIsIm1hYyI6IjZjYzJjN2IxNDg0NjVjZTI5MDIyZTVlOGVkYTgzZDc1MzZmYTQ3NjY1MGM5NmZjZDYzYTNmNmM2OTAxNzczY2IifQ%3D%3D; 591_new_session=eyJpdiI6IkxWY1VhejFMemZHSkVMd01WVUhFQ0E9PSIsInZhbHVlIjoiZzdhVlU2bk9LSjNnQVBHT09ES1pDSGo4Zk0rMlJINVFxYk12eVpVcFp1MFYwbXp1XC9HbkxFYUdVMTZPelMyTnZHT1NUWFVTOHgwN3ZzXC9oN1wvbkVOOVE9PSIsIm1hYyI6IjhjOTU4NjZiMDM4M2VjZWM3NmQ5MjBkYmQzZWI4ZjRjNWQ0NDQwNDJlZDM4NGU4MTRkODhmODVmYTM0NjdiYTkifQ%3D%3D",
    "X-CSRF-TOKEN": "OH3dybKd3yaw65OAZtcfIGeGR0y6MzhG9Lj3wNA4",
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"
}

res = requests.get(url2,headers=headers)

raw_data = res.json()


<Response [200]>


In [135]:
### 資料清洗 (ETL , 抓取欄位)
### (Extract-Transform-Load)

# area: "15"
# price: "26,300"
# title: "短租電梯定期免費打掃房間電梯二房一廳"
# post_id : 1234

def parsing_data(raw_data):

    dataList = []

    for ele in raw_data["data"]["data"]:
        
        # 拔掉車位 (看情況教)
        if(ele["kind_name"] == "車位"): continue

        area  = float(ele["area"])
        price = int(ele["price"].replace(",",""))
        dataList.append({
            "area"  : area,
            "price" : price,
            "title" : ele["title"],
            "post_id" : ele["post_id"],
            "url"   : "https://rent.591.com.tw/home/{}".format(ele["post_id"]),
            "price_per_area" : round(price / area , 2)
        })
        
    return dataList


# dataList
# parsing_data(raw_data)

In [ ]:
##### 抓取分頁 #1
    
# section 代號
# 中山 -> 3
# 信義 -> 7
# 松山 -> 4

### 先以 中山區 作測試
# https://rent.591.com.tw/home/search/rsList?is_format_data=1&is_new_list=1&type=1&section={}&searchtype=1&showMore=1&multiArea=10_20&firstRow={}&totalRows=993
api = "https://rent.591.com.tw/home/search/rsList?is_format_data=1&is_new_list=1&type=1&section={sec}&searchtype=1&showMore=1&multiArea=10_20&firstRow={row}&totalRows=993"
section = 3


headers = {
    "Host": "rent.591.com.tw",
    "Referer": "https://rent.591.com.tw/?multiArea=10_20&showMore=1&section=3&searchtype=1&firstRow=0&totalRows=993",
    "Cookie": "webp=1; PHPSESSID=g3u3tnc81fjrk7ighgv2mkr3m4; urlJumpIp=1; urlJumpIpByTxt=%E5%8F%B0%E5%8C%97%E5%B8%82; new_rent_list_kind_test=1; newUI=1; T591_TOKEN=g3u3tnc81fjrk7ighgv2mkr3m4; _ga=GA1.3.1649913125.1636598035; _gid=GA1.3.822631898.1636598035; _ga=GA1.4.1649913125.1636598035; _gid=GA1.4.822631898.1636598035; tw591__privacy_agree=0; _fbp=fb.2.1636598035774.332302313; _gat_UA-97423186-1=1; XSRF-TOKEN=eyJpdiI6IlpSOXZ1SVlCZHpzNWswdzQzYThLUFE9PSIsInZhbHVlIjoiTGdzaXFPblN4bUZmQ3U1Z3AzWG1MZUNrbDFiMjIxMlNDamhUaWg5SVBFUUNZOEU0SHlzK1Z3S21walVDWWpHZFR4QVFmbEdrZmRqQnZyd0s2aitkRUE9PSIsIm1hYyI6IjZjYzJjN2IxNDg0NjVjZTI5MDIyZTVlOGVkYTgzZDc1MzZmYTQ3NjY1MGM5NmZjZDYzYTNmNmM2OTAxNzczY2IifQ%3D%3D; 591_new_session=eyJpdiI6IkxWY1VhejFMemZHSkVMd01WVUhFQ0E9PSIsInZhbHVlIjoiZzdhVlU2bk9LSjNnQVBHT09ES1pDSGo4Zk0rMlJINVFxYk12eVpVcFp1MFYwbXp1XC9HbkxFYUdVMTZPelMyTnZHT1NUWFVTOHgwN3ZzXC9oN1wvbkVOOVE9PSIsIm1hYyI6IjhjOTU4NjZiMDM4M2VjZWM3NmQ5MjBkYmQzZWI4ZjRjNWQ0NDQwNDJlZDM4NGU4MTRkODhmODVmYTM0NjdiYTkifQ%3D%3D",
    "X-CSRF-TOKEN": "OH3dybKd3yaw65OAZtcfIGeGR0y6MzhG9Lj3wNA4",
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"
}

result = []


for i in range(5):
    firstRow = 30*i
    # print(firstRow)
    url = api.format(sec=section , row=firstRow)
    res = requests.get(url,headers=headers)
    raw_data = res.json()
    
    dataList = parsing_data(raw_data)
    
    result += dataList
    
    print("{} is ok.".format(url))
    
print("Done.")

In [ ]:
##### 抓取分頁 #2
import time
    
# section 代號
# 中山 -> 3
# 信義 -> 7
# 松山 -> 4

### 使用 for-loop 將三區域 一次抓
# https://rent.591.com.tw/home/search/rsList?is_format_data=1&is_new_list=1&type=1&section={}&searchtype=1&showMore=1&multiArea=10_20&firstRow={}&totalRows=993
api = "https://rent.591.com.tw/home/search/rsList?is_format_data=1&is_new_list=1&type=1&section={sec}&searchtype=1&showMore=1&multiArea=10_20&firstRow={row}&totalRows=993"
headers = {
    "Host": "rent.591.com.tw",
    "Referer": "https://rent.591.com.tw/?multiArea=10_20&showMore=1&section=3&searchtype=1&firstRow=0&totalRows=993",
    "Cookie": "webp=1; PHPSESSID=g3u3tnc81fjrk7ighgv2mkr3m4; urlJumpIp=1; urlJumpIpByTxt=%E5%8F%B0%E5%8C%97%E5%B8%82; new_rent_list_kind_test=1; newUI=1; T591_TOKEN=g3u3tnc81fjrk7ighgv2mkr3m4; _ga=GA1.3.1649913125.1636598035; _gid=GA1.3.822631898.1636598035; _ga=GA1.4.1649913125.1636598035; _gid=GA1.4.822631898.1636598035; tw591__privacy_agree=0; _fbp=fb.2.1636598035774.332302313; _gat_UA-97423186-1=1; XSRF-TOKEN=eyJpdiI6IlpSOXZ1SVlCZHpzNWswdzQzYThLUFE9PSIsInZhbHVlIjoiTGdzaXFPblN4bUZmQ3U1Z3AzWG1MZUNrbDFiMjIxMlNDamhUaWg5SVBFUUNZOEU0SHlzK1Z3S21walVDWWpHZFR4QVFmbEdrZmRqQnZyd0s2aitkRUE9PSIsIm1hYyI6IjZjYzJjN2IxNDg0NjVjZTI5MDIyZTVlOGVkYTgzZDc1MzZmYTQ3NjY1MGM5NmZjZDYzYTNmNmM2OTAxNzczY2IifQ%3D%3D; 591_new_session=eyJpdiI6IkxWY1VhejFMemZHSkVMd01WVUhFQ0E9PSIsInZhbHVlIjoiZzdhVlU2bk9LSjNnQVBHT09ES1pDSGo4Zk0rMlJINVFxYk12eVpVcFp1MFYwbXp1XC9HbkxFYUdVMTZPelMyTnZHT1NUWFVTOHgwN3ZzXC9oN1wvbkVOOVE9PSIsIm1hYyI6IjhjOTU4NjZiMDM4M2VjZWM3NmQ5MjBkYmQzZWI4ZjRjNWQ0NDQwNDJlZDM4NGU4MTRkODhmODVmYTM0NjdiYTkifQ%3D%3D",
    "X-CSRF-TOKEN": "OH3dybKd3yaw65OAZtcfIGeGR0y6MzhG9Lj3wNA4",
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"
}

dataDict = {
    "3" : [],
    "7" : [],
    "4" : []
}
mapping = {
    "3" : "中山區",
    "7" : "信義區",
    "4" : "松山區"
}


for section in [3,7,4]:
    
    for i in range(5):
        firstRow = 30*i
        url = api.format(sec=section , row=firstRow)
        
        time.sleep(0.3)
        
        res = requests.get(url,headers=headers)
        raw_data = res.json()

        dataList = parsing_data(raw_data)
        
        dataDict[str(section)] += dataList

        print("{} is ok.".format(url))
        
    print("{} 抓取完畢！".format(mapping[str(section)]))
    print("-"*80)
    
print("Done.")

In [136]:
# dataDict["4"]

In [137]:
# sum([ele["price"] for ele in dataDict["4"]])

In [138]:
# sorted(dataDict["4"] , key = lambda ele : ele["price_per_area"] , reverse=True)

In [132]:
##### 分析命題


for key in dataDict:
    section_name = mapping[key]
    print("『{}』 : ".format(section_name))
    
    # 1. 顯示 各區的 平均月租金 & 平均之坪數
    avg_price = sum([ ele["price"] for ele in dataDict[key]]) / len(dataDict[key])
    print("平均價位 :" , round(avg_price,3))
    
    avg_area  = sum([ ele["area"] for ele in dataDict[key]]) / len(dataDict[key])
    print("平均坪數 :" , round(avg_area,3))
    
    ###########################################################################
    
    
    # 2. 顯示 各區中 一坪單價最高之物件資料 , 需顯示該物件的 
    #    名稱 / 單位坪數價格 / 連結 , 可連回原本之網頁供參
    
    sorted_data = sorted(dataDict[key] , key = lambda ele : ele["price_per_area"] , reverse=True)
    big_element = sorted_data[0]
    # print(big_element)
    
    print("單價最高之物件 :" , big_element["title"])
    print("單價最高之物件一坪單價 :" , big_element["price_per_area"])
    print("單價最高之物件連結 : " , big_element["url"])

    print("-"*80)

『中山區』 : 
平均價位 : 24731.221
平均坪數 : 13.069
單價最高之物件 : 法式浪漫主義.唯美輕奢.懂享受不凡的自己
單價最高之物件一坪單價 : 3181.73
單價最高之物件連結 :  https://rent.591.com.tw/home/11668431
--------------------------------------------------------------------------------
『信義區』 : 
平均價位 : 27835.4
平均坪數 : 13.342
單價最高之物件 : MRT市政府•2BR•Ser.Apt.
單價最高之物件一坪單價 : 5201.72
單價最高之物件連結 :  https://rent.591.com.tw/home/11569231
--------------------------------------------------------------------------------
『松山區』 : 
平均價位 : 24961.858
平均坪數 : 13.904
單價最高之物件 : MRT小巨蛋•1BR•Ser.Apt.
單價最高之物件一坪單價 : 6923.08
單價最高之物件連結 :  https://rent.591.com.tw/home/11569362
--------------------------------------------------------------------------------
